In [2]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [3]:
from langchain.prompts import PromptTemplate

In [4]:
from langchain.output_parsers import PydanticOutputParser

In [5]:
from langchain_core.pydantic_v1 import BaseModel, Field, validator

In [6]:
from typing import List

In [7]:
class Reviews_Desired_Output_Structure(BaseModel):
    sentiment: List[str] = Field(
        description="Is it positive or negative?"
    )
    positive: List[str] = Field(
        description="What is positively highlighted about the book in the review?"
    )
    negative: List[str] = Field(
        description="What is negatively highlighted about the book in the review?"
    )

In [8]:
output_parser = PydanticOutputParser(
    pydantic_object=Reviews_Desired_Output_Structure
)

In [9]:
template_for_desired_output_with_parser = """
Here is a book review: {book_review}.

I want you to output three things about the review in a JSON dictionary:

{format_instructions}

"sentiment": Is it positive or negative?
"positive": What is positively highlighted about the book in the review?
"negative": What is negatively highlighted about the book in the review?

"""

In [10]:
prompt_template_with_parser = PromptTemplate(
    template=template_for_desired_output_with_parser,
    input_variables=["book_review"],
    partial_variables={
        "format_instructions": output_parser.get_format_instructions()
    }
)

In [12]:
user_input="this book is amazing. the character arc was great. the way the character evolves from a good person to a very bad person is very interesting"

In [15]:
question = prompt_template_with_parser.format(book_review=user_input)

In [17]:
from langchain_openai import OpenAI
llm=OpenAI()
response = llm(question)

/Users/anshulvirmani/.pyenv/versions/3.11.4/envs/environmentname1/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [18]:
formatted_output = output_parser.parse(response)

In [19]:
print(type(formatted_output))

<class '__main__.Reviews_Desired_Output_Structure'>


In [20]:
json_output = formatted_output.json()

In [21]:
import json

In [22]:
python_dict = json.loads(json_output)

In [23]:
print(python_dict)

{'sentiment': ['positive'], 'positive': ['amazing', 'great', 'interesting'], 'negative': ['bad']}


In [24]:
print(type(python_dict))

<class 'dict'>
